In [13]:
import pandas as pd
import numpy as np
import string

In [14]:
raw_13 = pd.read_csv('13_raw.csv')

In [15]:
raw_13.head()

,time,content,score,main,volume,vote,type
0,The Hush-Hush Reviewer wrote a review Nov 2020,Busy busy even with Covid so that might put so...,"<span class=""ui_bubble_rating bubble_40""></span>",Busy busy,661 contributions,242 helpful votes,Trip type: Traveled solo
1,Alex C wrote a review Nov 2020,Great shopping centre and plenty of bespoke sh...,"<span class=""ui_bubble_rating bubble_50""></span>",Great shopping,67 contributions,69 helpful votes,Trip type: Traveled as a couple
2,Nishamanttey wrote a review Oct 2020,The street runs through the city center. A lot...,"<span class=""ui_bubble_rating bubble_50""></span>",Hanging out,167 contributions,160 helpful votes,Trip type: Traveled solo
3,Malgorzata wrote a review Oct 2020,Buchanan Street is one of Glasgow's main comme...,"<span class=""ui_bubble_rating bubble_40""></span>",Great shopping street,NaN,905 helpful votes,NaN
4,g h wrote a review Sep 2020,Buchanan Street was beautiful we walked up and...,"<span class=""ui_bubble_rating bubble_50""></span>","beautiful street, the nicest we saw in Glasgow",83 contributions,1 helpful vote,NaN


In [16]:
raw_13.shape

(4035, 7)

In [17]:
raw_13.dtypes

time       object
content    object
score      object
main       object
volume     object
vote       object
type       object
dtype: object

In [18]:
raw_13.describe()

,time,content,score,main,volume,vote,type
count,4035,4035,4035,4035,3611,3569,3540
unique,4028,4034,5,3136,753,379,5
top,David M wrote a review May 2017,Busy shopping street. Pedestrianised with lots...,"<span class=""ui_bubble_rating bubble_50""></span>",Shopping,13 contributions,2 helpful votes,Trip type: Traveled as a couple
freq,2,2,2445,98,50,133,1380


In [19]:
username_time = raw_13['time'].str.split(' wrote a review ',expand=True)
username_time.columns = ['name','date']
raw_13['score'] = raw_13['score'].str.extract(r'(\d+)',expand=False)
raw_13['contribution'] = raw_13['volume'].str.extract(r'(\d+)',expand=False)
raw_13['vote'] = raw_13['vote'].str.extract(r'(\d+)',expand=False)
travel_type = raw_13['type'].str.split('Trip type: Traveled ',expand=True)
travel_type.columns = ['useless','travel_type']

In [20]:
raw_13 = pd.concat([username_time,raw_13,travel_type],axis=1)

In [21]:
raw_13.head()

,name,date,time,content,score,main,volume,vote,type,contribution,useless,travel_type
0,The Hush-Hush Reviewer,Nov 2020,The Hush-Hush Reviewer wrote a review Nov 2020,Busy busy even with Covid so that might put so...,40,Busy busy,661 contributions,242,Trip type: Traveled solo,661,,solo
1,Alex C,Nov 2020,Alex C wrote a review Nov 2020,Great shopping centre and plenty of bespoke sh...,50,Great shopping,67 contributions,69,Trip type: Traveled as a couple,67,,as a couple
2,Nishamanttey,Oct 2020,Nishamanttey wrote a review Oct 2020,The street runs through the city center. A lot...,50,Hanging out,167 contributions,160,Trip type: Traveled solo,167,,solo
3,Malgorzata,Oct 2020,Malgorzata wrote a review Oct 2020,Buchanan Street is one of Glasgow's main comme...,40,Great shopping street,NaN,905,NaN,NaN,NaN,NaN
4,g h,Sep 2020,g h wrote a review Sep 2020,Buchanan Street was beautiful we walked up and...,50,"beautiful street, the nicest we saw in Glasgow",83 contributions,1,NaN,83,NaN,NaN


In [22]:
raw_13 = raw_13.drop(['time','volume','type','useless',], axis=1)

In [23]:
raw_13.isnull().sum().sort_values(ascending=False)

travel_type     495
vote            466
contribution    424
main              0
score             0
content           0
date              0
name              0
dtype: int64

In [24]:
raw_13['contribution'] = raw_13['contribution'].fillna('0')
raw_13['contribution'] = raw_13['contribution'].astype(str).astype(int)
raw_13['vote'] = raw_13['vote'].fillna('0')
raw_13['vote'] = raw_13['vote'].astype(str).astype(int)
raw_13['travel_type'] = raw_13['travel_type'].fillna('Not Given')
raw_13['score'] = raw_13['score'].astype(str).astype(int)
raw_13['score'] = raw_13['score']/10
raw_13['date'] = pd.to_datetime(raw_13['date'],infer_datetime_format=True)

In [25]:
raw_13.isnull().sum().sort_values(ascending=False)

travel_type     0
contribution    0
vote            0
main            0
score           0
content         0
date            0
name            0
dtype: int64

### In some rare cases, the 'date of experience' is missing. Because the comments in TripAdvisor are sorted by time except for the homepage, and the time is only accurate to the month. Therefore, the 'date' is very close to or equal to the adjacent previous or next value. So, in this research, filled the missing 'date' with adjacent values from front to back.

### raw_01['date'] = raw_01['date'].fillna(method='ffill')

In [27]:
raw_13.shape

(4035, 8)

In [28]:
raw_13.dtypes

name                    object
date            datetime64[ns]
content                 object
score                  float64
main                    object
vote                     int32
contribution             int32
travel_type             object
dtype: object

In [29]:
raw_13.head()

,name,date,content,score,main,vote,contribution,travel_type
0,The Hush-Hush Reviewer,2020-11-01,Busy busy even with Covid so that might put so...,4.0,Busy busy,242,661,solo
1,Alex C,2020-11-01,Great shopping centre and plenty of bespoke sh...,5.0,Great shopping,69,67,as a couple
2,Nishamanttey,2020-10-01,The street runs through the city center. A lot...,5.0,Hanging out,160,167,solo
3,Malgorzata,2020-10-01,Buchanan Street is one of Glasgow's main comme...,4.0,Great shopping street,905,0,Not Given
4,g h,2020-09-01,Buchanan Street was beautiful we walked up and...,5.0,"beautiful street, the nicest we saw in Glasgow",1,83,Not Given


In [30]:
raw_13.describe()

,score,vote,contribution
count,4035.000000,4035.000000,4035.000000
mean,4.512763,51.468897,112.728129
std,0.690988,95.540164,164.098931
min,1.000000,0.000000,0.000000
25%,4.000000,5.000000,12.000000
50%,5.000000,18.000000,45.000000
75%,5.000000,55.000000,137.000000
max,5.000000,950.000000,991.000000


In [31]:
raw_13.to_csv('13.csv')

### Use the same method to process the other 14 raw data

In [32]:
a1 = pd.read_csv('01.csv')
a2 = pd.read_csv('02.csv')
a3 = pd.read_csv('03.csv')
a4 = pd.read_csv('04.csv')
a5 = pd.read_csv('05.csv')
a6 = pd.read_csv('06.csv')
a7 = pd.read_csv('07.csv')
a8 = pd.read_csv('08.csv')
a9 = pd.read_csv('09.csv')
a10 = pd.read_csv('10.csv')
a11 = pd.read_csv('11.csv')
a12 = pd.read_csv('12.csv')
a13 = pd.read_csv('13.csv')
a14 = pd.read_csv('14.csv')
a15 = pd.read_csv('15.csv')

In [33]:
a1 = a1.drop('Unnamed: 0', axis=1)
a2 = a2.drop('Unnamed: 0', axis=1)
a3 = a3.drop('Unnamed: 0', axis=1)
a4 = a4.drop('Unnamed: 0', axis=1)
a5 = a5.drop('Unnamed: 0', axis=1)
a6 = a6.drop('Unnamed: 0', axis=1)
a7 = a7.drop('Unnamed: 0', axis=1)
a8 = a8.drop('Unnamed: 0', axis=1)
a9 = a9.drop('Unnamed: 0', axis=1)
a10 = a10.drop('Unnamed: 0', axis=1)
a11 = a11.drop('Unnamed: 0', axis=1)
a12 = a12.drop('Unnamed: 0', axis=1)
a13 = a13.drop('Unnamed: 0', axis=1)
a14 = a14.drop('Unnamed: 0', axis=1)
a15 = a15.drop('Unnamed: 0', axis=1)

In [34]:
a1['site'] = 'Kelvingrove Art Gallery and Museum'
a2['site'] = 'The Riverside Museum of Transport and Travel'
a3['site'] = 'Glengoyne Distillery'
a4['site'] = 'Celtic Park'
a5['site'] = 'University of Glasgow'
a6['site'] = 'The Necropolis'
a7['site'] = 'Tennents Wellpark Brewery'
a8['site'] = 'The Clydeside Distillery'
a9['site'] = 'Ibrox Stadium'
a10['site'] = 'Glasgow Botanic Gardens'
a11['site'] = 'Glasgow Science Centre'
a12['site'] = 'Glasgow Cathedral'
a13['site'] = 'Buchanan Street'
a14['site'] = 'Pollok Country Park'
a15['site'] = 'The Tenement House'

In [35]:
site_frames = [a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15]
siteall = pd.concat(site_frames)

In [36]:
siteall.shape

(44033, 9)

In [37]:
siteall.isnull().sum().sort_values(ascending=False)

site            0
travel_type     0
contribution    0
vote            0
main            0
score           0
content         0
date            0
name            0
dtype: int64

In [38]:
siteall['date'] = pd.to_datetime(siteall['date'],infer_datetime_format=True)

In [39]:
siteall.dtypes

name                    object
date            datetime64[ns]
content                 object
score                  float64
main                    object
vote                     int64
contribution             int64
travel_type             object
site                    object
dtype: object

In [40]:
siteall.to_csv('top15.csv')